# Libreta práctica que unifica xlrd y postgresql
## Por: Jonathan Martiñón

# Librerías

In [15]:
# Importamos xlrd
try:
    import xlrd
except:
    !pip install xlrd
    import xlrd

# importamos psycopg2
try:
    import psycopg2
except:
    !pip install psycopg2-binary
    import psycopg2

# Funciones

In [47]:
def extrae_registro(registro):
    '''
    @Autor: Jonathan Martiñón

    @Descripción:   Función encargada de extraer valores
                    de un renglón de xlrd y eliminar vacíos.
    @Parámetros:

    lista           registro de elementos              LIST of xlrd.sheet.Cell

    @Devuelve:  
    
    new_registro    Lista con valores ya sin formato   LIST
                    de celdas

    @Ejemplo:

    # Cada elemento debe ser de tipo xlrd.sheet.Cell
    registro = [number:1.0, text:'Gómez', text:'García', text:'Eduardo']
    extrae_registro(registro)
    >>> [1, 'Gómez', 'García', 'Eduardo']
    
    
    registro = [number:4.0, empty:'', empty:'', text:'Sandra']
    extrae_registro(registro)
    >>> [4, 'default', 'default', 'Sandra']
    
    '''
    # Creamos una tupla auxiliar que almacenará el registro con el 
    # nuevo formato
    new_registro = ()
    
    # Recorremos cada elemento del registro
    for i, campo in enumerate(registro):
        
        # si existe valor en el campo
        if campo.value:
                
            #Sabemos que sólo hay enteros en id, por lo que
            try:
                # Intentaremos convertir el valor a entero
                new_registro+= (int(campo.value),)
            except:
                # Si no se puede, es una cadena de texto
                new_registro+= (campo.value,)
                
        # En caso de que el campo esté vacío
        else:
            new_registro+= ("default",)
    
    # Devolvemos el registro
    return new_registro

# Lectura de datos

In [16]:
# Realizamos la lectura del documento excel
documento = xlrd.open_workbook("./Documento interactivo para taller.xlsx")
documento

Ya conocemos el documento y las hojas presentes, pero de igual forma, mostramos cuáles son las hojas presentes

In [17]:
# Mostramos los nombres de las hojas
documento.sheet_names()

['Calificaciones', 'Alumnos', 'Materias', 'Profesores']

Debido a la presencia de datos nulos, trabajaremos con la tabla **'Alumnos'**

In [18]:
hoja_alumnos = documento.sheet_by_name("Alumnos")
hoja_alumnos

# Análisis

Identificamos a partir de qué punto comienzan nuestros registros

In [19]:
# inicio de cabeceras
i= 2
print(f"Renglón {i+1}: {hoja_alumnos.row(i)}")

Renglón 3: [text:'No. Cuenta', text:'Primer apellido', text:'Segundo Apellido', text:'Nombre(s)']


In [20]:
# Inicio de registros
i= 3
print(f"Renglón {i+1}: {hoja_alumnos.row(i)}")

Renglón 4: [number:1.0, text:'Gómez', text:'García', text:'Eduardo']


# Pre-procesamiento

In [21]:
# Generamos nuestra instrucción
instruccion = '''
INSERT INTO alumno (numero_de_cuenta, primer_apellido, segundo_apellido,nombre)
VALUES (%s,%s,%s,%s)
'''

Recopilaremos cada renglón presente en el registro

In [48]:
# Generamos una lista que contenga los registros
registros = []

# Sabemos que los registros abarcan toda la hoja
# A partir del elemento 3, por lo que ajustamos
# El ciclo
for i in range(3, hoja_alumnos.nrows):
    
    # Mandamos el renglón a nuestra función
    # para que pueda extraer y darle el formato que
    # necesitamos.
    reg_aux = extrae_registro(hoja_alumnos.row(i))
    
    # Posteriormente, lo agregamos en nuestra lista
    registros.append(reg_aux)

Verificamos que ya podemos aplicar un **executemany()**

In [49]:
print("Instrucción:\n",instruccion)
print("Valores:\n",registros)

Instrucción:
 
INSERT INTO alumno (numero_de_cuenta, primer_apellido, segundo_apellido,nombre)
VALUES (%s,%s,%s,%s)

Valores:
 [(1, 'Gómez', 'García', 'Eduardo'), (2, 'Ortega', 'default', 'Amelia'), (3, 'Martínez', 'Orduña', 'Erick'), (4, 'default', 'default', 'Sandra'), (5, 'Sandoval', 'Hernández', 'Antonio'), (6, 'Moreno', 'Corella', 'Andrea'), (7, 'Díaz', 'Castro', 'Bernardo'), (8, 'Cisneros', 'López', 'Akatzin'), (9, 'Tapia', 'default', 'Mauricio'), (10, 'Luna', 'Téllez', 'Ivana')]


# Procesamiento

In [52]:
# Nos conectamos a la base de datos
conexion = psycopg2.connect(database="taller_db", user="user_t", password="usuario", host="localhost", port ="5432")

In [53]:
# Iniciamos nuestro cursor
cursor=conexion.cursor()

In [54]:
# Ralizamos nuestro INSERT
cursor.executemany(instruccion, registros)

print("Datos subidos con éxito")

Datos subidos con éxito


In [55]:
# Verificamos que sí existan valores

# Realizamos una consulta
cursor.execute("SELECT * FROM alumno")

#Aplicando un primer fetchall
cursor.fetchall()

[(1,
  'Gómez               ',
  'García              ',
  'Eduardo                                 '),
 (2,
  'Ortega              ',
  'default             ',
  'Amelia                                  '),
 (3,
  'Martínez            ',
  'Orduña              ',
  'Erick                                   '),
 (4,
  'default             ',
  'default             ',
  'Sandra                                  '),
 (5,
  'Sandoval            ',
  'Hernández           ',
  'Antonio                                 '),
 (6,
  'Moreno              ',
  'Corella             ',
  'Andrea                                  '),
 (7,
  'Díaz                ',
  'Castro              ',
  'Bernardo                                '),
 (8,
  'Cisneros            ',
  'López               ',
  'Akatzin                                 '),
 (9,
  'Tapia               ',
  'default             ',
  'Mauricio                                '),
 (10,
  'Luna                ',
  'Téllez              ',
  'Iva

In [56]:
# Guardamos cambios
conexion.commit()

In [57]:
# Finalizamos la sesión

# Cerramos el cursor primero
cursor.close()
# Y Posteriormente cerramos la conexión
conexion.close()
# Un bonito mensaje final
print("La conexión se ha cerrado.\n\tGracias por tu visita (:")

La conexión se ha cerrado.
	Gracias por tu visita (:
